## CNN

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # X is a 4-D tensor: [batch_size, width, height, channels]
    # Angio images are 23x23 pixels, and have three color channel
    #input_layer = features["x"]
    input_layer = tf.reshape(features["x"], [-1, 23, 23, 1])
    print('input_layer: ', input_layer.shape)
    
    # Convolutional Layer #1
    # Computes 64 features using a 3x3 filter, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 23, 23, 3]
    # Output Tensor Shape: [batch_size, 23, 23, 64]
    conv1 = tf.layers.conv2d(
    inputs = input_layer,
    filters = 64,
    kernel_size = [3, 3],
    padding = "same")  
    print('conv1: ', conv1.shape)

    # Pooling Layer #1
    # Max pooling layer with a 3x3 filter and stride of 2
    # Input Tensor Shape: [batch_size, 23, 23, 64]
    # Output Tensor Shape: [batch_size, 11, 11, 64]
    pool1 = tf.layers.max_pooling2d(
    inputs = conv1,
    pool_size = [3, 3],
    strides = 2)
    print('pool1: ', pool1.shape)
    
    # Convolutional Layer #2
    # Computes 128 features using a 3x3 filter, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 11, 11, 64]
    # Output Tensor Shape: [batch_size, 11, 11, 128]
    conv2 = tf.layers.conv2d(
    inputs = pool1,
    filters = 128,
    kernel_size = [3, 3],
    padding = "same")
    print('conv2: ', conv2.shape)
    
    # Pooling Layer #2
    # Max pooling layer with a 3x3 filter and stride of 2
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 11, 11, 128]
    # Output Tensor Shape: [batch_size, 5, 5, 128]
    pool2  = tf.layers.max_pooling2d(
    inputs = conv2,
    pool_size = [3, 3],
    strides = 2)
    print('pool2: ', pool2.shape)
    
    # Convolutional Layer #3
    # Computes 256 features using a 3x3 filter and stride of 2, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 5, 5, 128]
    # Output Tensor Shape: [batch_size, 3, 3, 256]
    conv3 = tf.layers.conv2d(
    inputs = pool2,
    filters = 256,
    kernel_size = [3, 3],
    strides = 1,
    padding = "valid")
    print('conv3: ', conv3.shape)
    
    # Pooling Layer #3
    # Max pooling layer with a 3x3 filter
    # No padding is added.
    # Input Tensor Shape: [batch_size, 3, 3, 256]
    # Output Tensor Shape: [batch_size, 1, 1, 256]
    pool3 = tf.layers.max_pooling2d(
    inputs = conv3,
    pool_size = [3, 3],
    strides = 1)
    print('pool3: ', pool3.shape)
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 1, 1, 256]
    # Output Tensor Shape: [batch_size, 1 * 1 * 256]
    pool2flat = tf.reshape(pool3, [-1, 1 * 1 * 256])
    print('pool2flat: ', pool2flat.shape)

    # Dense Layer
    # Densely connected layer with 256 neurons
    # Input Tensor Shape: [batch_size, 1 * 1 * 256]
    # Output Tensor Shape: [batch_size, 256]    
    dense = tf.layers.dense(
    inputs = pool2flat,
    units = 256)
    print('dense: ', dense.shape)
    
    # Logits layer
    # Input Tensor Shape: [batch_size, 256]
    # Output Tensor Shape: [batch_size, 2]
    logits = tf.layers.dense(
    inputs = dense,
    units = 2)
    print('logits: ', logits.shape)
    
    classes = tf.argmax(input = logits, axis = 1, name = "class")
    print('classes: ', classes.shape)
    probabilities = tf.nn.softmax(logits, name = "softmax_tensor")
    print('probabilities: ', probabilities.shape)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": classes,
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`
        "probabilities": probabilities
    }
    print('predictions: ', len(predictions))
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    #onehot_labels = tf.concat([1 - labels, labels], axis = 1)
    #print('labels: ', labels.shape)
    #print('onehot labels: ', onehot_labels.shape)
    #softmax_logits = tf.nn.softmax(logits)
    #loss = tf.losses.log_loss(labels = onehot_labels, predictions = softmax_logits)  
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    print('...Loss...')
    
    # Configure the Training Op (for TRAIN mode)
    if mode ==tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
    
    print('labels: ', labels.shape)
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels, predictions = predictions["classes"]),
        "precision": tf.metrics.precision(
            labels = labels, predictions = predictions["classes"]),
        "recall": tf.metrics.recall(
            labels = labels, predictions = predictions["classes"]),
        "ROC": tf.metrics.auc(
            labels = labels, predictions = predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = eval_metric_ops)

## Train the model

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load angio data
%run preprocess_angio_data.ipynb

In [ ]:
# Create the Estimator
angio_classifier_1 = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/angio_model6")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=3000)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_std},
    y=y_train,
    #x={"x": train_data},
    #y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
angio_classifier_1.train(
    input_fn=train_input_fn,
    steps=12000,
    hooks=[logging_hook])

In [ ]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_std},
    y=y_test,
    #x={"x": eval_data},
    #y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = angio_classifier_1.evaluate(input_fn=eval_input_fn)
print(eval_results)